In [1]:
%%time
!pip install -q --upgrade pip fastai

CPU times: user 28.7 ms, sys: 6.55 ms, total: 35.2 ms
Wall time: 1.75 s


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *

In [4]:
NUMBER_OF_SAMPLES = 50_000
SIZE = 224 // 8 
MAGNIFICATION = 8
INSETS = 2
THRESHOLD = 0.8

NUMBER_OF_SAMPLES * SIZE * SIZE

39200000

# Creating test data

In [5]:
from PIL import ImageDraw as PILImageDraw
from PIL import Image as PILImage

def create_data():
    return np.random.standard_normal(
        size=(NUMBER_OF_SAMPLES, SIZE, SIZE)) > THRESHOLD

def get_label(data):
    return float(np.sum(data, axis=(0,1)))

def render_image(data):
    img = PILImage.new('RGB', (SIZE*MAGNIFICATION, SIZE*MAGNIFICATION), color = (255,255, 255))
    draw = PILImageDraw.Draw(img)

    for (row_index, row) in enumerate(data):
        for (col_index, value) in enumerate(row):
            x0, y0 = row_index*MAGNIFICATION+INSETS, col_index*MAGNIFICATION+INSETS
            x1, y1 = ((row_index+1)*MAGNIFICATION-INSETS)+1, \
                     ((col_index+1)*MAGNIFICATION-INSETS)+1

            if value: 
                draw.rectangle([(x0, y0), (x1, y1)], fill=(255, 153, 0))  
                
    return Image(pil2tensor(img, np.float32).div(255))

class MyImageItemList(ImageList):
    def __init__(self, path, items, **kwargs):
        super().__init__(items, **kwargs)
        self.path = path
    
    def get(self, i):
        return render_image(self.items[i])

In [6]:
%%time
dt = create_data()
iil = MyImageItemList(Path('.'), items=dt)
ll = iil.split_by_rand_pct(0.2)
ll.label_from_func(get_label)
#data_bunch = ll.databunch(bs=20)
#data_bunch = data_bunch.normalize(imagenet_stats)

CPU times: user 2.2 s, sys: 204 ms, total: 2.4 s
Wall time: 2.4 s


In [7]:
#data_bunch.label_list

In [8]:
#data_bunch.show_batch(2, figsize=(7, 7))

# Create CNN

In [ ]:
runs = []

for (architecture, bs) in [(models.resnet101, 100), (models.resnet50, 150), (models.resnet34, 260), (models.resnet18, 380)]:
    runs += [(1, 5e-2, 384, bs, architecture,  str(architecture).split()[1])] 

losses = []
#runs = [] # Skip for now
for (epochs, lr, size, bs, architecture,  label) in runs:
    print(f'Starting {label}.')
    #data_bunch = ImageDataBunch.from_lists(path='.',fnames=fn_list, labels=label_list, 
    #                                       bs=bs, size=size, ds_tfms=tfms).normalize(imagenet_stats)
    data_bunch = ll.databunch(bs=bs)
    print(bs)
    data_bunch = data_bunch.normalize(imagenet_stats)
    model = cnn_learner(data_bunch, architecture)
    model.fit_one_cycle(epochs, slice(lr))
    model.recorder.plot_losses()
    plt.yscale('log')
    plt.title(table)
    losses += [(label, model.recorder.losses, model.recorder.val_losses)]

Starting resnet101.
100


epoch,train_loss,valid_loss,time


In [ ]:
def plot_results(losses, title):
    colors = plt.cm.rainbow(np.linspace(0,1,len(runs)))
    plt.figure(figsize=(16,8))
    plt.ylim(0, 3000)
    for idx, (label, train_loss, val_loss) in enumerate(losses):
        color = colors[idx]
        plt.plot(train_loss, color=color, linestyle="-.", label=label+' train')
        plt.plot(val_loss,   color=color, linestyle="-", label=label+' val')
    plt.legend()
    plt.title(title)


plot_results(filter(lambda x: 'resnet18' in x[0], losses), 'Different Sizes')
plt.xlim(0,250)
plot_results(list(filter(lambda x: 'x48' in x[0], losses))[:3], 'Different Sizes')
plt.xlim(0,250)
plt.xlim(0,500)
plot_results(filter(lambda x: '256' in x[0], losses), 'Resnet vs Densnet')
# Apparently smaller is better?
plt.yscale('log')
plot_results(losses, "All")
plt.yscale('log')
#for (label, train_loss, valid_loss) in losses:
#    plot_results([(label, train_loss, valid_loss)], label)
plt.xlim(0,250)
plt.yscale('log')

labels, train_losses, val_losses = [], [], []
for (label, train_loss, val_loss) in losses:
    labels += [label]
    train_losses += [train_loss[-1].item()]
    val_losses   += [val_loss[-1].item()]

results = pd.DataFrame()
results['label'] = np.array(labels, dtype=np.str)

results['train_loss'] = np.array(train_losses, dtype=np.float32)
results['val_loss'] = np.array(val_losses, dtype=np.float32)
results.sort_values(by='val_loss', inplace=True)
display(results)  